In [1]:
from pathlib import Path
from typing import Optional, Dict, List, Tuple
from pydantic import BaseModel, Field, field_validator
from mercantile import Tile, tile
import geopandas as gpd

import os, sys

PROJECT_PATH = Path(os.getcwd()).resolve().parent.parent
print(f'Treating "{PROJECT_PATH}" as `project_path`')

sys.path.append(str(PROJECT_PATH))
from preprocessing.location import Location

UNIVERSE_NAME = 'caprecon3'
UNIVERSE_PATH = PROJECT_PATH / 'src/streetTransformer/data/universes/' / UNIVERSE_NAME
locations_gdf = gpd.read_feather(UNIVERSE_PATH / 'locations.feather').rename(columns={'NODEID':'location_id'})
locations_gdf


Treating "/Users/jon/Documents/Employment/2025 VIDA Lab/streetTransformer" as `project_path`
Treating "/Users/jon/Documents/Employment/2025 VIDA Lab/streetTransformer" as `project_path`


,location_id,geometry,StreetNames
0,2196,POINT (933671.717 134443.939),"[PHILIP AVE, AMBOY RD, ALBEE AVE]"
1,2198,POINT (933986.135 134580.324),"[AMBOY RD, ANNADALE RD]"
2,2258,POINT (934313.579 134957.373),"[ANNADALE RD, POILLON AVE]"
3,2259,POINT (934246.551 134504.402),"[AMBOY RD, COTTONTAIL CT]"
4,2267,POINT (934463.52 135150.49),"[FURMAN ST, ANNADALE RD, ELY CT]"
...,...,...,...
5337,9066652,POINT (1049106.447 198061.631),"[99 AVE, PEDESTRIAN TUNNEL]"
5338,9066653,POINT (1049078.112 198148.214),"[LIRR MAIN LINE, PEDESTRIAN TUNNEL]"
5339,9067013,POINT (993785.29 191933.947),[MYRTLE AVE]
5340,9067031,POINT (997535.312 191956.318),[WILLOUGHBY AVE]


In [15]:
# import mercantile

#LocationImagery(locations_gdf)
# coord = locations_gdf.to_crs('4326')['geometry'][0]
# temp_li = LocationImagery(centroid=(coord.x, coord.y))

# #[mercantile.bounds(t) for t in temp_li.tile_grid]
# temp_li.
# #LocationImagery()
# # #temp_li
# # mercantile.tile(coord.x, coord.y, 20)
test_location_series = locations_gdf.to_crs('4326').loc[0]
# test_location = Location(
#     location_id=test_location_series['location_id'],
#     universe_name='caprecon3',
#     crossstreets=test_location_series['StreetNames'],
#     centroid=test_location_series['geometry'],
#     years=list(range(2006, 2025, 2))
# )


documents_in_buffer = {}
for row in locations_gdf.to_crs('4326').itertuples():
    test_location = Location(
        location_id=row.location_id,
        universe_name='caprecon3',
        crossstreets=row.StreetNames,
        centroid=row.geometry,
        years=list(range(2006, 2025, 2))
    )

    documents_in_buffer[row.location_id] = test_location.documents

KeyboardInterrupt: 

In [ ]:
import pandas as pd
pd.concat(documents_in_buffer).shape # so we hvae like 10k projects within 1000 feet of a location.

(10452, 9)

In [3]:
# Let's redo with the square:
documents_in_bbox = {}
import tqdm
n_locations = locations_gdf.shape[0]
for row in tqdm.tqdm(locations_gdf.to_crs('4326').itertuples(), total=n_locations, ):
    test_location = Location(
        location_id=row.location_id,
        universe_name='caprecon3',
        crossstreets=row.StreetNames,
        centroid=row.geometry,
        years=list(range(2006, 2025, 2))
    )

    documents_in_bbox[row.location_id] = test_location.documents

100%|██████████| 5342/5342 [05:32<00:00, 16.06it/s]


In [ ]:
import pandas as pd
pd.concat(documents_in_bbox).shape # 1645

pd.concat(documents_in_bbox).groupby(level=0).count() # 1,147

pd.concat(documents_in_bbox).groupby(level=1).count() # only 249 projects


,year,borough,name,project_id,full_paths,relative_paths,coords,geometry
3,5,5,5,5,5,5,5,5
5,9,9,9,9,9,9,9,9
8,6,6,6,6,6,6,6,6
9,5,5,5,5,5,5,5,5
10,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...
929,19,19,19,19,19,19,19,19
930,12,12,12,12,12,12,12,12
935,22,22,22,22,22,22,22,22
938,4,4,4,4,4,4,4,4
